In [85]:
import numpy as np
import csv
import datetime

In [86]:
class Visitor:
    def __init__(self, name, id, money):
        self.name = name
        self.id = id
        self.money = money

    @property
    def id(self):
        return self._id
    
    @id.setter
    def id(self, id):
        if id < 0:
            raise ValueError("id must be positive")
        self._id = id
    
    @property
    def money(self):
        return self._money
    
    @money.setter
    def money(self, money):
        if money < 0:
            raise ValueError("money must be positive")
        self._money = money

    def pay(self, amount):
        if amount > self.money:
            raise ValueError("not enough money")
        self.money -= amount

    def __str__(self):
        return f"Visitor: {self.name}, id: {self.id}, money: {self.money}"

In [87]:
class Training:
    def __init__(self, name, id, price):
        self.name = name
        self.id = id
        self.price = price
    
    @property
    def id(self):
        return self._id
    
    @id.setter
    def id(self, id):
        if id < 0:
            raise ValueError("id must be positive")
        self._id = id

    @property
    def price(self):
        return self._price
    
    @price.setter
    def price(self, price):
        if price < 0:
            raise ValueError("price must be positive")
        self._price = price
    
    def __str__(self):
        return f"Training: {self.name}, id: {self.id}, price: {self.price}"

In [88]:
class TrainingInfo:
    def __init__(self, date, visitor_id, training_id):
        self.date = datetime.datetime.strptime(date, '%Y-%m-%d')
        self.visitor_id = visitor_id
        self.training_id = training_id
    
    def __str__(self):
        return f"TrainingInfo: {self.date}, visitor_id: {self.visitor_id}, training_id: {self.training_id}"
    

In [89]:
visitors = dict()
trainings = dict()
with open("visitors.csv", "r") as file:
    reader = csv.reader(file)
    next(file)
    for row in reader:
        visitors[int(row[1])] = Visitor(row[0], int(row[1]), float(row[2]))

with open("training.csv", "r") as file:
    next(file)
    reader = csv.reader(file)
    for row in reader:
        trainings[int(row[1])] = Training(row[0], int(row[1]), float(row[2]))


In [90]:
infos = []
with open("info.csv", "r") as file:
    next(file)
    reader = csv.reader(file)
    for row in reader:
        infos.append(TrainingInfo(row[0], int(row[1]), int(row[2])))

In [98]:
#a
trainings_per_visitor = {}
for info in infos:
    if info.training_id in trainings_per_visitor:
        trainings_per_visitor[info.training_id] += 1
    else:
        trainings_per_visitor[info.training_id] = 1




In [99]:
print(trainings_per_visitor)

{1: 2, 2: 2, 3: 1}


In [100]:
#b
def count_payment(time_a, time_b, visitor_id):
    payment = 0
    count = 0
    for info in infos:
        if info.visitor_id == visitor_id and datetime.datetime.strptime(time_a, '%Y-%m-%d') <= info.date and info.date <= datetime.datetime.strptime(time_b, '%Y-%m-%d'):
            if count == 0:    
                payment += int(trainings[info.training_id].price)
            else:
                payment += int(trainings[info.training_id].price) * 0.6
            count += 1
    
    visitors[visitor_id].pay(payment)
    return payment, visitors[visitor_id].money

In [101]:
count_payment("2019-01-01", "2023-01-31", 2)

(15.0, 70.0)

In [102]:
#c
def visitor_expenses(name):
    visitor_id = None
    for visitor in visitors.values():
        if visitor.name == name:
            visitor_id = visitor.id
            break
    if visitor_id == None:
        raise ValueError("visitor not found")
    
    payment = {}
    dates = set()
    for info in infos:
        if info.visitor_id == visitor_id:
            if info.training_id in payment:
                if info.date in dates:
                    payment[trainings[info.training_id].name] += int(trainings[info.training_id].price) * 0.6
                else:
                    payment[trainings[info.training_id].name] += int(trainings[info.training_id].price)
            else:
                if info.date in dates:
                    payment[trainings[info.training_id].name] = int(trainings[info.training_id].price) * 0.6
                else:
                    payment[trainings[info.training_id].name] = int(trainings[info.training_id].price)
            dates.add(info.date)
    return payment

In [103]:
visitor_expenses("Sam")

{'Pull-ups': 9, 'Jumping-Jacks': 6.0}